In [5]:
import pandas as pd
from matplotlib import pyplot as plt
import sklearn.datasets

def get_iris_df():
    ds = sklearn.datasets.load_iris()
    df = pd.DataFrame(ds['data'],columns=ds['feature_names'])
    code_species_map = dict(zip(range(3), ds['target_names']))
    df['species'] = [code_species_map[c] for c in ds['target']]
    return df

In [6]:
df = get_iris_df()

In [13]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# decision tree

> 특정 기준(질문)에 따라 데이터를 구분하는 모델


결정 트리를 학습하는 방법은 다음과 같습니다.

1. 데이터를 보고 분류를 잘하는 최적의 특징값과 기준을 찾아냅니다.
2. 분류가 잘되었는지 평가하는 방법은 다양합니다. 가장 흔히 사용하는 방법은 정보 이득과 지니 불순도입니다.
3. 이렇게 찾아낸 최적의 특징값과 기준은 결정 트리의 최상위 노드가 됩니다.
4. 이 작업을 계속 반복해 데이터를 나눕니다.
5. 더 이상 데이터를 나눌 필요가 없어지거나 미리 정해놓은 트리의 높이에 도달하면 멈춥니다. 각 단말 노드에 있는 데이터 분포가 해당 노드의 확률분포가 됩니다. 예를 들어 잎사귀 노드에 범주 1의 데이터가 30개, 범주 2의 데이터가 20개 있다면 범주가 1/2일 확률을 각각 60%, 40%로 계산합니다.

- 결정 트리의 구체적인 학습 방법을 알아 두면 과적합과 같은 문제에 대처하는 데 도움이 됩니다.
- 예를 들어 트리의 최대 높이를 너무 크게 설정하면 잎사귀 노드마다 아주 적은 데이터 표본만 포함하게 됩니다. 결과적으로 100%에 가까운 예측 확률을 얻게 되는데 실제로 정확도가 올라간 것이 아니라 과적합이 일어난 겁니다.
- 라이브러리의 기본 설정을 그대로 사용하면 이런 문제가 흔히 발생하기 때문에 세부 매개변수를 조절하려면 학습 원리를 알고 있어야 합니다.

`과적합 : 머신러닝 모델이 학습 데이터에 너무 치우쳐서 일반화하지 못하게 된 것`

```python
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(train[indep_cols],train.breed)
predictions = clf.predict(test[indep_cols])
```

In [18]:
from sklearn.tree import DecisionTreeClassifier   
from sklearn.datasets import load_iris    
from sklearn.model_selection import train_test_split

In [50]:
# Load iris data

iris_data=load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, test_size=0.2, random_state=24)

In [51]:
# Create DecisionTreeClassifier
clf= DecisionTreeClassifier(max_depth=5) 

In [52]:
# Train clf
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5)

In [53]:
print("훈련 세트 정확도: {:.3f}".format(clf.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(clf.score(X_test, y_test)))

훈련 세트 정확도: 0.992
테스트 세트 정확도: 0.933
